In [2]:
import numpy as np
import os
import librosa
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential 
from keras.layers import  Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
file_path="audio"
max_pad_length=40
random_state=42

ModuleNotFoundError: No module named 'librosa'

In [4]:
def test_wavfile(path="audio/go/0d53e045_nohash_1.wav"):
    
    sample_rate, samples = wavfile.read(path)
    frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)
    plt.pcolormesh(times, frequencies, spectrogram)
    plt.imshow(spectrogram)
    plt.ylabel('Frequecy [HZ]')
    plt.xlabel('Time [sec]')
    plt.show()

In [5]:
def convert_wave_to_mfcc(filePath):
    wav, sr =librosa.load(filePath, mono = True, sr=None)
    mfcc = librosa.feature.mfcc(wav, sr=sr)
    ##padding
    mfcc_feature = np.pad(mfcc, pad_width=((0, 0), (0, max_pad_length - mfcc.shape[1])), mode='constant')
    return mfcc_feature

In [6]:
def get_labels(path=file_path):
    return os.listdir(path)

In [7]:
def save_mfcc_files(path=file_path):
    labels = get_labels(path)
    #print(labels)
    for label in labels:
        mfcc_features=[]
        audfiles=[path+'/'+label+'/'+aud for aud in os.listdir(path+'/'+label)]
        for audio in audfiles:
            mfcc_features.append(convert_wave_to_mfcc(audio))
        #print(len(mfcc_vectors))
        np.save(label+'.npy', mfcc_features)

In [30]:
def generate_training_testing_set(path=file_path,test_size=.4):
    labels = get_labels(path)
    #features=[]
    features=np.load(labels[0]+'.npy')
    classes=np.zeros(features.shape[0])
    #for i,label in zip(range(0,len(labels)-1),labels):
    for i in range(1,len(labels)):
        x=np.load(labels[i]+'.npy')
        features=np.vstack((features,x))
        classes=np.append(classes, np.full(x.shape[0], fill_value= (i)))
    return train_test_split(features, classes, test_size= test_size, random_state=random_state, shuffle=True)

In [36]:
X_train, X_test, y_train, y_test = generate_training_testing_set()
print(X_train.shape)
model =Sequential()
model.add(Conv2D(64,kernel_size=(2,2),activation='relu',input_shape=(20,40,1)))
model.add(MaxPooling2D(pool_size=(2,2),strides=1,padding='same'))
model.add(Dropout(.2))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(.2))
model.add(Dense(11,activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


(14574, 20, 40)


In [38]:
model.fit(X_train.reshape(X_train.shape[0],20,40,1), to_categorical(y_train), batch_size=128, epochs=100, verbose=1,validation_split=0.3 )

Train on 10201 samples, validate on 4373 samples
Epoch 1/100
10201/10201 [==============================] - 3s 265us/step - loss: 1.5830 - acc: 0.4377 - val_loss: 1.6175 - val_acc: 0.4558
Epoch 2/100
10201/10201 [==============================] - 3s 255us/step - loss: 1.5430 - acc: 0.4463 - val_loss: 1.4222 - val_acc: 0.5106
Epoch 3/100
10201/10201 [==============================] - 3s 253us/step - loss: 1.5045 - acc: 0.4618 - val_loss: 1.4120 - val_acc: 0.5422
Epoch 4/100
10201/10201 [==============================] - 3s 253us/step - loss: 1.4504 - acc: 0.4747 - val_loss: 1.5025 - val_acc: 0.5054
Epoch 5/100
10201/10201 [==============================] - 3s 252us/step - loss: 1.4167 - acc: 0.4911 - val_loss: 1.4002 - val_acc: 0.5362
Epoch 6/100
10201/10201 [==============================] - 3s 252us/step - loss: 1.3798 - acc: 0.5009 - val_loss: 1.3832 - val_acc: 0.5413
Epoch 7/100
10201/10201 [==============================] - 3s 252us/step - loss: 1.3273 - acc: 0.5190 - val_loss: 1.3

In [1]:
y_predicted=model.predict(X_test.reshape(X_test.shape[0],20,40,1))
b = np.zeros_like(y_test)
b[range(len(y_predicted)), y_predicted.argmax(1)] = 1
print(b[0])

NameError: name 'model' is not defined